In [89]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import os
import numpy as np
import pandas as pd


#### The EuroSAT dataset


The dataset is from https://github.com/phelber/EuroSAT. For a reference, see the following papers:
- Eurosat: A novel dataset and deep learning benchmark for land use and land cover classification. Patrick Helber, Benjamin Bischke, Andreas Dengel, Damian Borth. IEEE Journal of Selected Topics in Applied Earth Observations and Remote Sensing, 2019.
- Introducing EuroSAT: A Novel Dataset and Deep Learning Benchmark for Land Use and Land Cover Classification. Patrick Helber, Benjamin Bischke, Andreas Dengel. 2018 IEEE International Geoscience and Remote Sensing Symposium, 2018.


#### Import the data

In [90]:
def load_eurosat_data():
    data_dir = 'data/'
    x_train = np.load(os.path.join(data_dir, 'x_train.npy'))
    y_train = np.load(os.path.join(data_dir, 'y_train.npy'))
    x_test  = np.load(os.path.join(data_dir, 'x_test.npy'))
    y_test  = np.load(os.path.join(data_dir, 'y_test.npy'))
    return (x_train, y_train), (x_test, y_test)

(x_train, y_train), (x_test, y_test) = load_eurosat_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

#### Build the neural network model

In [91]:
def get_new_model(input_shape):

    model = Sequential([
        Conv2D(16, (3,3), activation='relu', padding='same', input_shape=input_shape, name='conv_1'),
        Conv2D(8, (3,3), activation='relu', padding='same', name='conv_2'),
        MaxPooling2D((8,8), name='pool_1'),
        Flatten(),
        Dense(32, activation='relu', name='dense_1'),
        Dense(10, activation='softmax', name='dense_2')
    ])
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model
    

#### Compile and evaluate the model

In [92]:
model = get_new_model(x_train[0].shape)

In [93]:
def get_test_accuracy(model, x_test, y_test):
    """Test model classification accuracy"""
    test_loss, test_acc = model.evaluate(x=x_test, y=y_test, verbose=0)
    print('accuracy: {acc:0.3f}'.format(acc=test_acc))

In [94]:
model.summary()
get_test_accuracy(model, x_test, y_test)

Model: "sequential_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 64, 64, 16)        448       
_________________________________________________________________
conv_2 (Conv2D)              (None, 64, 64, 8)         1160      
_________________________________________________________________
pool_1 (MaxPooling2D)        (None, 8, 8, 8)           0         
_________________________________________________________________
flatten_31 (Flatten)         (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                16416     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                330       
Total params: 18,354
Trainable params: 18,354
Non-trainable params: 0
_________________________________________________

#### Create checkpoints to save model during training, with a criterion


In [95]:
def get_checkpoint_every_epoch():

    return ModelCheckpoint(
        filepath = 'checkpoints_every_epoch/checkpoint_{epoch:03d}',
        frequency='epoch',
        save_weights_only=True, 
        verbose=1)


def get_checkpoint_best_only():
 
    return ModelCheckpoint(
        filepath = 'checkpoints_best_only/checkpoint',
        frequency='epoch',
        save_best_only=True,
        monitor='val_accuracy',
        save_weights_only=True, 
        verbose=1)

In [96]:
def get_early_stopping():

    
    return EarlyStopping(monitor='val_accuracy', patience=3)

In [97]:
checkpoint_every_epoch = get_checkpoint_every_epoch()
checkpoint_best_only = get_checkpoint_best_only()
early_stopping = get_early_stopping()

#### Train model using the callbacks


In [98]:
callbacks = [checkpoint_every_epoch, checkpoint_best_only, early_stopping]
model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test), callbacks=callbacks)

Train on 4000 samples, validate on 1000 samples
Epoch 1/50
3968/4000 [============================>.] - ETA: 0s - loss: 2.1096 - accuracy: 0.1920
Epoch 00001: saving model to checkpoints_every_epoch/checkpoint_001

Epoch 00001: val_accuracy improved from -inf to 0.32700, saving model to checkpoints_best_only/checkpoint
4000/4000 [==============================] - 86s 21ms/sample - loss: 2.1064 - accuracy: 0.1933 - val_loss: 1.7541 - val_accuracy: 0.3270
Epoch 2/50
3968/4000 [============================>.] - ETA: 0s - loss: 1.5583 - accuracy: 0.3982
Epoch 00002: saving model to checkpoints_every_epoch/checkpoint_002

Epoch 00002: val_accuracy improved from 0.32700 to 0.39200, saving model to checkpoints_best_only/checkpoint
4000/4000 [==============================] - 87s 22ms/sample - loss: 1.5572 - accuracy: 0.3985 - val_loss: 1.4861 - val_accuracy: 0.3920
Epoch 3/50
3968/4000 [============================>.] - ETA: 0s - loss: 1.3715 - accuracy: 0.4851
Epoch 00003: saving model to ch

#### Create new instance of model and load on both sets of weights


In [99]:
checkpoint_path = 'checkpoints_every_epoch\checkpoint_{epoch:03d}'
checkpoint_best_only = 'checkpoints_best_only\checkpoint'

def get_model_last_epoch(model):

    model.load_weights(tf.train.latest_checkpoint(checkpoint_dir='checkpoints_every_epoch'))
    return model
    
    
def get_model_best_epoch(model):

    
    model.load_weights('checkpoints_best_only/checkpoint')
    return model


In [101]:
model_last_epoch = get_model_last_epoch(get_new_model(x_train[0].shape))
model_best_epoch = get_model_best_epoch(get_new_model(x_train[0].shape))
print('Model with last epoch weights:')
get_test_accuracy(model_last_epoch, x_test, y_test)
print('')
print('Model with best epoch weights:')
get_test_accuracy(model_best_epoch, x_test, y_test)

Model with last epoch weights:
accuracy: 0.665

Model with best epoch weights:
accuracy: 0.669


#### Load, from scratch, a model trained on the EuroSat dataset.


In [102]:
def get_model_eurosatnet():

    return load_model('models/EuroSatNet.h5')
    
    
    

In [103]:
model_eurosatnet = get_model_eurosatnet()
model_eurosatnet.summary()
get_test_accuracy(model_eurosatnet, x_test, y_test)

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 64, 64, 16)        448       
_________________________________________________________________
conv_2 (Conv2D)              (None, 64, 64, 16)        6416      
_________________________________________________________________
pool_1 (MaxPooling2D)        (None, 32, 32, 16)        0         
_________________________________________________________________
conv_3 (Conv2D)              (None, 32, 32, 16)        2320      
_________________________________________________________________
conv_4 (Conv2D)              (None, 32, 32, 16)        6416      
_________________________________________________________________
pool_2 (MaxPooling2D)        (None, 16, 16, 16)        0         
_________________________________________________________________
conv_5 (Conv2D)              (None, 16, 16, 16)      